In [ ]:
!pip install scikit-learn numpy pandas matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

# Load Emotion_final.csv dataset
emotion_data = pd.read_csv('/kaggle/input/emotions-in-text/Emotion_final.csv')

# Clean the text in the emotion data
def clean_text(text):
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text

emotion_data['Text'] = emotion_data['Text'].apply(clean_text)

# Load synthetic_data.csv dataset
synthetic_data = pd.read_csv('/kaggle/input/emotional-dataset-v2/synthetic_dataset.csv')
synthetic_data['Text'] = synthetic_data['Text'].apply(clean_text)

# Concatenate the original and synthetic datasets
combined_data = pd.concat([emotion_data, synthetic_data], ignore_index=True)
combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

# Hyperparameters
num_clients = 3  # Increase for more diverse federated learning
epochs = 100  # Keep, but implement early stopping based on validation loss
learning_rate = 5e-4  # Start with a lower learning rate, consider scheduling decay
subset_size = 1000  # Increase for more diverse data sampling
batch_size = 128  # Larger batch size for faster convergence
embedding_dim = 256  # Increase for capturing more complex patterns
hidden_dim = 256  # Increase for more model capacity


# Federated Learning - Split data for each client
clients_data = []
for _ in range(num_clients):
    client_data = combined_data.sample(frac=0.2, random_state=42).reset_index(drop=True).head(subset_size)
    clients_data.append(client_data)

# Encoding the labels
label_encoder = LabelEncoder()
for i in range(num_clients):
    clients_data[i]['Emotion'] = label_encoder.fit_transform(clients_data[i]['Emotion'])

# Tokenizer function for splitting words
def tokenize(text):
    return text.split()

# Building a vocabulary
all_words = set(word for text in combined_data['Text'] for word in tokenize(text))
word_to_index = {word: i+1 for i, word in enumerate(all_words)}  # Reserve index 0 for padding

# Custom Dataset class
# class TextDataset(Dataset):
#     def __init__(self, texts, labels, max_length=128):
#         self.texts = texts.tolist()
#         self.labels = labels.tolist()
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         label = int(self.labels[idx])
#         tokenized = [word_to_index.get(word, 0) for word in tokenize(text)]
#         padding = [0] * (self.max_length - len(tokenized))
#         input_ids = tokenized[:self.max_length] + padding if len(tokenized) < self.max_length else tokenized[:self.max_length]
#         return {
#             'input_ids': torch.tensor(input_ids, dtype=torch.long),
#             'labels': torch.tensor(label, dtype=torch.long)
#         }

class TextDataset(Dataset):
    def __init__(self, texts, labels, word_to_index, max_length=128):
        self.texts = texts.tolist()  # Ensure the texts are in list format
        self.labels = labels.tolist()  # Ensure labels are in list format
        self.word_to_index = word_to_index  # Ensure that word_to_index is passed
        self.max_length = max_length  # Ensure max_length is assigned properly

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = int(self.labels[idx])

        # Tokenize the text
        tokenized = [self.word_to_index.get(word, 0) for word in tokenize(text)]

        # Apply padding
        padding = [0] * (self.max_length - len(tokenized))  # Add padding
        input_ids = tokenized[:self.max_length] + padding if len(tokenized) < self.max_length else tokenized[:self.max_length]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# BiGRU model definition
class BiGRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, max_length):
        super(BiGRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=0)
        self.bigru = nn.GRU(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, h = self.bigru(x)
        h = torch.cat((h[-2, :, :], h[-1, :, :]), dim=1)  # Concatenate final forward and backward hidden states
        return self.fc(h)

# Train each client locally
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = []
train_loss_history = [[] for _ in range(num_clients)]
train_acc_history = [[] for _ in range(num_clients)]

# for i in range(num_clients):
#     # Model and optimizer setup
#     model = BiGRUModel(len(word_to_index), embedding_dim, hidden_dim, len(label_encoder.classes_), max_length=128).to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         correct = 0
#         total = 0
#         for batch in train_loader:
#             input_ids = batch['input_ids'].to(device)
#             labels = batch['labels'].to(device)

#             optimizer.zero_grad()
#             outputs = model(input_ids)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()

#             _, predicted = torch.max(outputs, dim=1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#         avg_loss = total_loss / len(train_loader)
#         accuracy = correct / total
#         train_loss_history[i].append(avg_loss)
#         train_acc_history[i].append(accuracy)
#         print(f"Client {i}, Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {accuracy:.4f}")

#     models.append(model)

# Create DataLoader for each client's data
for i in range(num_clients):
    # Create the dataset for the client
    train_dataset = TextDataset(clients_data[i]['Text'], clients_data[i]['Emotion'], word_to_index=word_to_index)
    
    # Create DataLoader for this client's dataset
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Model and optimizer setup
    model = BiGRUModel(len(word_to_index), embedding_dim, hidden_dim, len(label_encoder.classes_), max_length=128).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = total_loss / len(train_loader)
        accuracy = correct / total
        train_loss_history[i].append(avg_loss)
        train_acc_history[i].append(accuracy)
        print(f"Client {i}, Epoch {epoch + 1}, Loss: {avg_loss}, Accuracy: {accuracy:.4f}")

    models.append(model)


# Aggregation Step: Averaging the weights of the models for a global model
global_state_dict = {}
for key in models[0].state_dict().keys():
    global_state_dict[key] = sum(model.state_dict()[key] for model in models) / num_clients

global_model = BiGRUModel(len(word_to_index), embedding_dim, hidden_dim, len(label_encoder.classes_), max_length=128).to(device)
global_model.load_state_dict(global_state_dict)

# Evaluate the global model
def evaluate_model(model, dataset):
    model.eval()
    data_loader = DataLoader(dataset, batch_size=batch_size)
    correct, total = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# original_test_dataset = TextDataset(emotion_data['Text'], label_encoder.transform(emotion_data['Emotion']))
# synthetic_test_dataset = TextDataset(synthetic_data['Text'], label_encoder.transform(synthetic_data['Emotion']))
# overall_accuracy = (evaluate_model(global_model, original_test_dataset) + evaluate_model(global_model, synthetic_test_dataset)) / 2
# print(f"Overall Test Accuracy: {overall_accuracy:.4f}")

# Create test datasets with the 'word_to_index' parameter
original_test_dataset = TextDataset(emotion_data['Text'], label_encoder.transform(emotion_data['Emotion']), word_to_index=word_to_index)
synthetic_test_dataset = TextDataset(synthetic_data['Text'], label_encoder.transform(synthetic_data['Emotion']), word_to_index=word_to_index)

# Evaluate the global model
overall_accuracy = (evaluate_model(global_model, original_test_dataset) + evaluate_model(global_model, synthetic_test_dataset)) / 2
print(f"Overall Test Accuracy: {overall_accuracy:.4f}")


# Plot the training loss and accuracy
plt.figure(figsize=(12, 6))
for i in range(num_clients):
    plt.subplot(2, num_clients, i+1)
    plt.plot(range(epochs), train_loss_history[i], label=f'Client {i} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Client {i} Training Loss')
    plt.legend()

    plt.subplot(2, num_clients, num_clients+i+1)
    plt.plot(range(epochs), train_acc_history[i], label=f'Client {i} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title(f'Client {i} Training Accuracy')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Forecast emotions for the test dataset (simulating future dataset)
def forecast(model, dataset, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    data_loader = DataLoader(dataset, batch_size=batch_size)  # Use DataLoader for batching
    with torch.no_grad():  # Disable gradient calculation
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)  # Get input data
            outputs = model(input_ids)  # Get model predictions
            _, predicted = torch.max(outputs, dim=1)  # Get the index of the max logit
            predictions.extend(predicted.cpu().numpy())  # Store predictions
    return np.array(predictions)  # Return predictions as a numpy array

# Simulated future dataset (for demonstration, we will use a random sample from the combined data)
simulated_future_texts = combined_data.sample(frac=0.1, random_state=42)['Text']
simulated_future_labels = label_encoder.transform(combined_data.sample(frac=0.1, random_state=42)['Emotion'])

# Use the TextDataset class to process the simulated future dataset
simulated_future_dataset = TextDataset(simulated_future_texts, simulated_future_labels, word_to_index=word_to_index)

# Forecast emotions for the simulated future dataset
predictions = forecast(global_model, simulated_future_dataset, device)
predicted_emotions = label_encoder.inverse_transform(predictions)  # Convert predictions back to emotion labels
print("Predicted Emotions for Simulated Test Dataset:", predicted_emotions)

# Evaluate performance on the simulated test dataset with ground truth
test_true_emotions = label_encoder.inverse_transform(simulated_future_labels)  # True emotions for the test set
accuracy = np.mean(predicted_emotions == test_true_emotions)  # Calculate accuracy
precision = precision_score(test_true_emotions, predicted_emotions, average='weighted')  # Precision score
recall = recall_score(test_true_emotions, predicted_emotions, average='weighted')  # Recall score
f1 = f1_score(test_true_emotions, predicted_emotions, average='weighted')  # F1 score
cm = confusion_matrix(test_true_emotions, predicted_emotions)  # Confusion matrix

# Print the evaluation metrics
print(f"Forecast Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(cm)

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
            xticklabels=True, yticklabels=True)

plt.title('Confusion Matrix Heatmap')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
import torch

def membership_inference_attack_biGRU(global_model, original_data, synthetic_data, word_to_index, max_len=128, device='cuda'):
    global_model.eval()  # Set the model to evaluation mode

    def get_max_prob(data, labels, word_to_index, max_len, device):
        max_probs = []

        for text in data:
            # Convert text to tensor indices
            tokenized = [word_to_index.get(word, 0) for word in text.split()]
            padding = [0] * (max_len - len(tokenized))
            input_ids = tokenized[:max_len] + padding if len(tokenized) < max_len else tokenized[:max_len]
            text_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension

            # Perform inference
            with torch.no_grad():
                outputs = global_model(text_tensor)
                probs = torch.softmax(outputs, dim=1)  # Get the probabilities for each class
                max_probs.append(probs.max().item())  # Get the maximum probability

        return np.array(max_probs)

    # Get the maximum probabilities for the original and synthetic data
    original_max_probs = get_max_prob(original_data['Text'], original_data['Emotion'], word_to_index, max_len, device)
    synthetic_max_probs = get_max_prob(synthetic_data['Text'], synthetic_data['Emotion'], word_to_index, max_len, device)

    # Combine the probabilities with labels for membership inference
    labels = np.concatenate([np.ones(len(original_max_probs)), np.zeros(len(synthetic_max_probs))])
    scores = np.concatenate([original_max_probs, synthetic_max_probs])

    # Calculate the AUC (Area Under the Curve) for the membership inference attack
    auc = roc_auc_score(labels, scores)
    print(f"Membership Inference Attack AUC on the Global BiGRU model: {auc:.4f}")

# Perform the membership inference attack on the Global BiGRU model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
membership_inference_attack_biGRU(global_model, emotion_data, synthetic_data, word_to_index, max_len=128, device=device)


In [ ]:
def membership_inference_attack_local_biGRU(local_models, clients_data, synthetic_data, word_to_index, max_len=128, device='cuda'):
    def get_max_prob(model, data, word_to_index, max_len, device):
        model.eval()  # Set the model to evaluation mode
        max_probs = []

        for text in data['Text']:
            # Tokenize text and convert it to tensor indices
            tokenized = [word_to_index.get(word, 0) for word in text.split()]
            padding = [0] * (max_len - len(tokenized))
            input_ids = tokenized[:max_len] + padding if len(tokenized) < max_len else tokenized[:max_len]
            text_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension

            # Perform inference
            with torch.no_grad():
                outputs = model(text_tensor)
                probs = torch.softmax(outputs, dim=1)  # Get the probabilities for each class
                max_probs.append(probs.max().item())  # Get the maximum probability

        return np.array(max_probs)

    # Perform the attack on each client's local model
    for i, (model, client_data) in enumerate(zip(local_models, clients_data)):
        print(f"Evaluating Membership Inference Attack on Client {i}'s Local BiGRU Model...")

        # Get maximum probabilities for client data (original) and synthetic data
        client_original_max_probs = get_max_prob(model, client_data, word_to_index, max_len, device)
        synthetic_max_probs = get_max_prob(model, synthetic_data, word_to_index, max_len, device)

        # Combine the probabilities with labels for membership inference
        labels = np.concatenate([np.ones(len(client_original_max_probs)), np.zeros(len(synthetic_max_probs))])
        scores = np.concatenate([client_original_max_probs, synthetic_max_probs])

        # Calculate AUC for the membership inference attack
        auc = roc_auc_score(labels, scores)
        print(f"Client {i} - Membership Inference Attack AUC: {auc:.4f}")

# Perform the Membership Inference Attack for each local BiGRU model
membership_inference_attack_local_biGRU(models, clients_data, synthetic_data, word_to_index, max_len=128, device=device)


In [ ]:
from sklearn.preprocessing import label_binarize  # Import label_binarize
from sklearn.metrics import roc_auc_score

# Adjusted generate_linkage_data function for BiGRU model
def generate_linkage_data_biGRU(models, clients_data, word_to_index, max_len=128, device='cuda'):
    all_predictions = []
    all_labels = []

    for i, client_data in enumerate(clients_data):
        # Convert client's text and labels to dataset using TextDataset
        dataset = TextDataset(client_data['Text'], client_data['Emotion'], word_to_index)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        client_predictions = []
        client_labels = []

        with torch.no_grad():
            for batch in data_loader:
                input_ids = batch['input_ids'].to(device)
                outputs = models[i](input_ids)  # Forward pass for each client’s BiGRU model
                probabilities = torch.softmax(outputs, dim=1)

                client_predictions.extend(probabilities.cpu().numpy())
                client_labels.extend([i] * len(probabilities))  # Use the client index as the label

        all_predictions.extend(client_predictions)
        all_labels.extend(client_labels)

    return np.array(all_predictions), np.array(all_labels)


# Generate linkage attack data with BiGRU models
all_predictions, all_labels = generate_linkage_data_biGRU(models, clients_data, word_to_index, max_len=128, device=device)

# Convert labels to one-hot encoding for AUC calculation
all_labels_one_hot = label_binarize(all_labels, classes=list(range(num_clients)))

# Calculate AUC for each client
auc_scores = []

for i in range(num_clients):
    # Calculate AUC for the current client (one-vs-rest)
    auc = roc_auc_score(all_labels_one_hot[:, i], all_predictions[:, i])
    auc_scores.append(auc)
    print(f"AUC for Client {i}: {auc:.4f}")

# Calculate macro-average AUC (average AUC across all clients)
macro_auc = np.mean(auc_scores)
print(f"Macro-Average AUC: {macro_auc:.4f}")

# Evaluate performance on the simulated test dataset with ground truth
test_true_emotions = label_encoder.inverse_transform(simulated_future_labels)
accuracy = np.mean(predicted_emotions == test_true_emotions)
precision = precision_score(test_true_emotions, predicted_emotions, average='weighted')
recall = recall_score(test_true_emotions, predicted_emotions, average='weighted')
f1 = f1_score(test_true_emotions, predicted_emotions, average='weighted')
cm = confusion_matrix(test_true_emotions, predicted_emotions)

print(f"Forecast Test Accuracy: {accuracy:.4f}")
print(f"Forecast Test Precision: {precision:.4f}")
print(f"Forecast Test Recall: {recall:.4f}")
print(f"Forecast Test F1 Score: {f1:.4f}")
print("Forecast Confusion Matrix:")
print(cm)